# 参数管理

- 访问参数，用于调试，诊断和可视化
- 参数初始化
- 在不同模型组件间共享参数

In [23]:
'''具有隐藏层的多层感知机'''
import torch
from torch import nn
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2505],
        [0.1299]], grad_fn=<AddmmBackward0>)

## 1. 参数访问

当通过sequential类定义模型时，可以通过索引访问模型任意层。每层的参数都在属性中

In [24]:
'''检查第二个全连接层的参数'''
print(net[2].state_dict())
print(net[2].weight)
print(type(net[2].weight.data)) # weights中有两个属性，可以通过data访问存有权重的张量

OrderedDict({'weight': tensor([[ 0.3294,  0.1147,  0.2975,  0.1485,  0.2110, -0.0629,  0.2663, -0.3516]]), 'bias': tensor([0.0438])})
Parameter containing:
tensor([[ 0.3294,  0.1147,  0.2975,  0.1485,  0.2110, -0.0629,  0.2663, -0.3516]],
       requires_grad=True)
<class 'torch.Tensor'>


### 1.1 目标参数

参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [25]:
print(type(net[2].bias)) # 符合对象，包含值，梯度
print(net[2].bias) # 符合对象，包含值，梯度
print(net[2].bias.data) # 访问存有权重值的张量

net[2].weight == None # 还没有进行反向传播，所以目前还有梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0438], requires_grad=True)
tensor([0.0438])


False

### 1.2 一次性访问所有参数

In [32]:
'''访问全连接层的参数，与，访问所有层的参数'''
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) # 使用*号将列表元素解包为单独的参数传递给print函数
print(*[(name, param.shape) for name, param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [35]:
'''另一种访问网络参数的方法'''
net.state_dict()['2.bias'].data # 访问2层偏置值

tensor([0.0438])